In [ ]:
#pip install websockets

In [ ]:
#pip install nest_asyncio

In [ ]:
#%autoawait asyncio

In [1]:
import pyaudio
import wave
import asyncio
from pydub import AudioSegment
import json
import json
import websockets
import smtplib
import ssl
from email.message import EmailMessage
from vosk import Model, KaldiRecognizer
from datetime import datetime

# Set the audio parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100  # Sample rate (samples per second)
RECORD_SECONDS = 5  # Duration of the recording in seconds
OUTPUT_FILENAME_TEMPLATE = "meeting_{}.wav"
loop_counter = 1

In [2]:
# Initialize PyAudio
async def startRecording(stop_flag):
    print("coming in")
    audio = pyaudio.PyAudio()

    loop_counter =1
    while not stop_flag.is_set():
        
        print(stop_flag.is_set())
            # Create an audio stream
        stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=1024)

        print(f"Recording loop {loop_counter} - {RECORD_SECONDS} seconds...")

        frames = []

            # Record audio
        for _ in range(0, int(RATE / 1024 * RECORD_SECONDS)):
            data = stream.read(1024)
            frames.append(data)

        print(f"Finished recording loop {loop_counter}.")

            # Stop and close the audio stream
        stream.stop_stream()
        stream.close()

            # Save the recorded audio to a WAV file
        output_filename = OUTPUT_FILENAME_TEMPLATE.format(loop_counter)
        with wave.open(output_filename, 'wb') as wf:
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(audio.get_sample_size(FORMAT))
                wf.setframerate(RATE)
                wf.writeframes(b''.join(frames))

        print(f"Audio saved to {output_filename}")

            # Increment the loop counter
        loop_counter += 1
        await asyncio.sleep(1)
    print("Recording stoppped")

In [3]:
def convertAudioToText():
    print("avuna",stop_flag.is_set())
    FRAME_RATE = 16000
    CHANNELS=1
    model = Model(model_name="vosk-model-en-us-0.22")
    # For a smaller download size, use model = Model(model_name="vosk-model-small-en-us-0.15")
    rec = KaldiRecognizer(model, FRAME_RATE)
    rec.SetWords(True)

    out = ''
    for i in range(1, loop_counter+1):  # Assuming n is the total number of recordings
        # Construct the filename
        filename = f"meeting_{i}.wav"

        # Load the audio file
        wav = AudioSegment.from_wav(filename)

        # Set the number of channels and frame rate
        wav = wav.set_channels(CHANNELS)
        wav = wav.set_frame_rate(FRAME_RATE)

        rec.AcceptWaveform(wav.raw_data)
        result = rec.Result()
        text = json.loads(result)["text"]
        out = out + '\n' + text
        print(text)
        return out

In [4]:
def summarizer():
    for i in range(1, 8):
        print('.')
    print("Summarizing Finsihed")
    return "Summarizing Finsihed"

In [5]:
def sendMail(emails, content):
    print('sending nmailaaa', emails)
    # Define email sender and receiver
    email_sender = 'kothapallipriyanka05@gmail.com'
    email_password = 'ibfk lefu zzfg gcjw'
    email_receiver = 'amalvalar@gmail.com'

    current_date = datetime.now().strftime("%Y-%m-%d")

    subject = "Meeting Agenda dated on " + current_date
    body = content

    em = EmailMessage()
    em['from'] = email_sender
    em['to'] = emails
    em['subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()
    # context.load_verify_locations('/Users/apple/Downloads/Meeting-Minder/venv/lib/python3.8/site-packages/certifi/cacert.pem')  # Replace with the actual path to cacert.pem

    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())

In [6]:
async def on_message(websocket, path, stop_flag):
    async for message in websocket:
        print(f"Received message from extension: {message}")
        
        # Process the message from the extension as needed
        # Parse the JSON string into a Python dictionary
        json_data = json.loads(message)

        # Access the value associated with the "action" key
        action_value = json_data['action']
        print(action_value)
        summarized_content=''
        # Send feedback to the extension
        if action_value =="startRecording":
            asyncio.create_task(startRecording(stop_flag)) 
        elif action_value =="stopRecording":
            stop_flag.set()
            convertAudioToText()
            response = {"message": "stopped"}
            await websocket.send(json.dumps(response))
            summarized_content =summarizer()
            response = {"message": "summarized"}
            await websocket.send(json.dumps(response))
        else:
            sendMail(json_data['email'], summarized_content)
            response = {"message": "mailSent"}
            await websocket.send(json.dumps(response))
        
stop_flag = asyncio.Event()

start_server = websockets.serve(lambda ws, path: on_message(ws, path, stop_flag), "localhost", 8774)

await start_server

Received message from extension: {"action":"startRecording"}
startRecording
coming in
False
Recording loop 1 - 5 seconds...
Finished recording loop 1.
Audio saved to meeting_1.wav
False
Recording loop 2 - 5 seconds...
Finished recording loop 2.
Audio saved to meeting_2.wav
Received message from extension: {"action":"stopRecording"}
stopRecording
avuna True
players need not the number of dimensions of the end
.
.
.
.
.
.
.
Summarizing Finsihed
Recording stoppped


In [ ]:
pip install pyaudio SpeechRecognition

In [ ]:
import speech_recognition as sr

def convert_live_audio_to_text():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Say something...")

        try:
            while True:
                audio_data = recognizer.listen(source, timeout=None)
                text = recognizer.recognize_google(audio_data)
                print("Text:", text)
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Error with the speech recognition service: {e}")

if __name__ == "__main__":
    convert_live_audio_to_text()


In [ ]:
import smtplib
import ssl
from email.message import EmailMessage
import os

# Set proxy environment variables if applicable
os.environ['http_proxy'] = 'http://your_proxy_address:your_proxy_port'
os.environ['https_proxy'] = 'http://your_proxy_address:your_proxy_port'


# Define email sender and receiver
email_sender = 'amalvalar@gmail.com'
email_password = 'cqay zvbz iwbx gbnf'
email_receiver = 'kothapallipriyanka05@gmail.com'

from email.message import EmailMessage
import ssl
import smtplib
email_sender ="amalvalar@gmail.com"
email_password = 'cqay zvbz iwbx gbnf'
email_receiver="amalvalar@gmail.com"
subject="Dont forget ti subscribe"
body='''Welcome to learn with esprit
'''
em = EmailMessage()
em['from']=email_sender
em['to']= email_receiver
em['subject']=subject
em.set_content (body)
context=ssl.create_default_context

with smtplib.SMTP_SSL("smtp.gmail.com",587) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receiver, em.as_string())